In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
Loss: 0.2534756660461426
Epoch 2/10
Loss: 0.14498403668403625
Epoch 3/10
Loss: 0.09527823328971863
Epoch 4/10
Loss: 0.06174764409661293
Epoch 5/10
Loss: 0.04807869717478752
Epoch 6/10
Loss: 0.03501637279987335
Epoch 7/10
Loss: 0.02258460968732834
Epoch 8/10
Loss: 0.018968263640999794
Epoch 9/10
Loss: 0.013071208260953426
Epoch 10/10
Loss: 0.009879383258521557


In [ ]:


import requests

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
response = requests.get(url)

with open('processed.cleveland.data', 'wb') as file:
    file.write(response.content)
# Load dataset
column_names = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'
]
data = pd.read_csv('processed.cleveland.data', header=None, names=column_names, na_values='?')

# Handle missing values by dropping rows with NaN values
data.dropna(inplace=True)

# Features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network architecture
num_features = X_train.shape[1]
num_classes = len(set(y))  # Number of unique target classes

# WTA Model
model_wta = models.Sequential()
model_wta.add(layers.InputLayer(input_shape=(num_features,)))
model_wta.add(layers.Dense(128, activation='relu'))
model_wta.add(layers.Dense(64, activation='relu'))
model_wta.add(layers.Dense(num_classes, activation='softmax'))

# Define WTA training function
# Custom training function for WTA
def train_wta(model, X_train, y_train, epochs, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        for i in range(len(X_train)):
            x = tf.convert_to_tensor([X_train[i]], dtype=tf.float32)  # Ensure x is a batch of size 1
            y = tf.convert_to_tensor([y_train[i]], dtype=tf.int32)

            with tf.GradientTape() as tape:
                predictions = model(x, training=True)
                loss = loss_fn(y, predictions)

                # Get the index of the neuron with the maximum activation (winner neuron)
                winner_neuron = tf.argmax(predictions, axis=-1)

                # Convert winner_neuron to an integer tensor for indexing
                winner_neuron_index = tf.cast(winner_neuron, tf.int32)

                # Extract weights of the last layer
                weights = model.layers[-1].weights[0]

                # Select weights of the winning neuron
                selected_weights = tf.gather(weights, winner_neuron_index, axis=1)

            # Compute gradients
            gradients = tape.gradient(loss, model.trainable_variables)

            # Apply gradients
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        print(f'Loss: {loss.numpy()}')


# Train the WTA model
train_wta(model_wta, X_train, y_train, epochs=10, learning_rate=0.001)


In [ ]:
# Custom training function for LTA
model_lta = models.Sequential()
model_lta.add(layers.InputLayer(input_shape=(num_features,)))
model_lta.add(layers.Dense(128, activation='relu'))
model_lta.add(layers.Dense(64, activation='relu'))
model_lta.add(layers.Dense(num_classes, activation='softmax'))
def train_lta(model, X_train, y_train, epochs, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        for i in range(len(X_train)):
            x = tf.convert_to_tensor([X_train[i]], dtype=tf.float32)
            y = tf.convert_to_tensor([y_train[i]], dtype=tf.int32)

            with tf.GradientTape() as tape:
                predictions = model(x, training=True)
                loss = loss_fn(y, predictions)

                # Get the index of the neuron with the minimum activation
                loser_neuron = tf.argmin(predictions, axis=-1)[0].numpy()

                # Get the weights of the last layer
                weights = model.layers[-1].weights[0]

                # Extract weights for the losing neuron
                selected_weights = tf.gather(weights, loser_neuron, axis=1)

            gradients = tape.gradient(loss, [selected_weights])
            optimizer.apply_gradients(zip(gradients, [selected_weights]))
        print(f'Loss: {loss.numpy()}')

In [ ]:
# Example input data
input_data = pd.DataFrame({
    'age': [50],
    'sex': [1],
    'cp': [2],
    'trestbps': [130],
    'chol': [250],
    'fbs': [0],
    'restecg': [1],
    'thalach': [170],
    'exang': [0],
    'oldpeak': [1.0],
    'slope': [2],
    'ca': [0],
    'thal': [2]
})

# Normalize the input data
input_data_scaled = scaler.transform(input_data.values)

# Predict using WTA model
predictions_wta = model_wta.predict(input_data_scaled)
predicted_class_wta = tf.argmax(predictions_wta, axis=-1).numpy()

# Predict using LTA model
predictions_lta = model_lta.predict(input_data_scaled)
predicted_class_lta = tf.argmax(predictions_lta, axis=-1).numpy()

print(f'Predicted Class using WTA: {predicted_class_wta}')
print(f'Predicted Class using LTA: {predicted_class_lta}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Class using WTA: [0]
Predicted Class using LTA: [1]


In [ ]:
# Example input data
input_data = pd.DataFrame({
    'age': [50],
    'sex': [1],
    'cp': [2],
    'trestbps': [130],
    'chol': [250],
    'fbs': [0],
    'restecg': [1],
    'thalach': [170],
    'exang': [0],
    'oldpeak': [1.0],
    'slope': [2],
    'ca': [0],
    'thal': [2]
})

# Normalize the input data
input_data_scaled = scaler.transform(input_data)

# Predict using WTA model
predictions_wta = model_wta.predict(input_data_scaled)
predicted_class_wta = tf.argmax(predictions_wta, axis=-1).numpy()

# Predict using LTA model
predictions_lta = model_lta.predict(input_data_scaled)
predicted_class_lta = tf.argmax(predictions_lta, axis=-1).numpy()

print(f'Predicted Class using WTA: {predicted_class_wta}')
print(f'Predicted Class using LTA: {predicted_class_lta}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted Class using WTA: [0]
Predicted Class using LTA: [3]


In [ ]:
# Example input data
input_data = pd.DataFrame({
    'age': [50],
    'sex': [1],
    'cp': [2],
    'trestbps': [130],
    'chol': [250],
    'fbs': [0],
    'restecg': [1],
    'thalach': [170],
    'exang': [0],
    'oldpeak': [1.0],
    'slope': [2],
    'ca': [0],
    'thal': [2]
})

# Normalize the input data
input_data_scaled = scaler.transform(input_data)

# Predict using WTA model
predictions_wta = model_wta.predict(input_data_scaled)
predicted_class_wta = tf.argmax(predictions_wta, axis=-1).numpy()[0]

# Predict using LTA model
predictions_lta = model_lta.predict(input_data_scaled)
predicted_class_lta = tf.argmax(predictions_lta, axis=-1).numpy()[0]

print(f'Predicted Class using WTA: {predicted_class_wta}')
print(f'Predicted Class using LTA: {predicted_class_lta}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Class using WTA: 0
Predicted Class using LTA: 1


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
# Example input data
input_data = pd.DataFrame({
    'age': [10],
    'sex': [1],
    'cp': [2],
    'trestbps': [130],
    'chol': [250],
    'fbs': [0],
    'restecg': [1],
    'thalach': [170],
    'exang': [0],
    'oldpeak': [1.0],
    'slope': [2],
    'ca': [0],
    'thal': [2]
})

# Normalize the input data
input_data_scaled = scaler.transform(input_data)

# Predict using WTA model
predictions_wta = model_wta.predict(input_data_scaled)
predicted_class_wta = tf.argmax(predictions_wta, axis=-1).numpy()[0]

# Predict using LTA model
predictions_lta = model_lta.predict(input_data_scaled)
predicted_class_lta = tf.argmax(predictions_lta, axis=-1).numpy()[0]

print(f'Predicted Class using WTA: {predicted_class_wta}')
print(f'Predicted Class using LTA: {predicted_class_lta}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Class using WTA: 0
Predicted Class using LTA: 4


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
import numpy as np

# Define a function to generate random input data within reasonable ranges for each feature
def generate_random_input():
    return pd.DataFrame({
        'age': [np.random.randint(30, 80)],
        'sex': [np.random.randint(0, 2)],
        'cp': [np.random.randint(0, 4)],
        'trestbps': [np.random.randint(90, 200)],
        'chol': [np.random.randint(150, 400)],
        'fbs': [np.random.randint(0, 2)],
        'restecg': [np.random.randint(0, 2)],
        'thalach': [np.random.randint(70, 210)],
        'exang': [np.random.randint(0, 2)],
        'oldpeak': [np.random.uniform(0, 6)],
        'slope': [np.random.randint(0, 3)],
        'ca': [np.random.randint(0, 4)],
        'thal': [np.random.randint(0, 3)]
    })

# List to store the input data sets
input_data_sets = []

while len(input_data_sets) < 10:
    input_data = generate_random_input()
    input_data_scaled = scaler.transform(input_data.values)

    # Predict using WTA model
    predictions_wta = model_wta.predict(input_data_scaled)
    predicted_class_wta = tf.argmax(predictions_wta, axis=-1).numpy()

    # Predict using LTA model
    predictions_lta = model_lta.predict(input_data_scaled)
    predicted_class_lta = tf.argmax(predictions_lta, axis=-1).numpy()

    # Check if WTA predicts class 1 and LTA predicts various classes
    if predicted_class_wta[0] == 1:
        input_data['predicted_class_wta'] = predicted_class_wta[0]
        input_data['predicted_class_lta'] = predicted_class_lta[0]
        input_data_sets.append(input_data)

# Display the generated input data sets
for idx, data in enumerate(input_data_sets):
    print(f"Set {idx + 1}:")
    print(data)
    print("\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Generate random data for 1000 samples
num_samples = 1000

# Define ranges for features
age = np.random.randint(20, 80, size=num_samples)
sex = np.random.randint(0, 2, size=num_samples)  # 0 or 1
cp = np.random.randint(0, 4, size=num_samples)   # Chest pain type
trestbps = np.random.randint(90, 200, size=num_samples)  # Resting blood pressure
chol = np.random.randint(150, 400, size=num_samples)     # Serum cholesterol
fbs = np.random.randint(0, 2, size=num_samples)  # Fasting blood sugar
restecg = np.random.randint(0, 2, size=num_samples)  # Resting electrocardiographic results
thalach = np.random.randint(60, 200, size=num_samples)  # Maximum heart rate
exang = np.random.randint(0, 2, size=num_samples)  # Exercise induced angina
oldpeak = np.random.uniform(0, 6, size=num_samples)  # Depression induced by exercise
slope = np.random.randint(0, 3, size=num_samples)  # Slope of the peak exercise ST segment
ca = np.random.randint(0, 5, size=num_samples)  # Number of major vessels colored by fluoroscopy
thal = np.random.randint(0, 4, size=num_samples)  # Thalassemia

# Create DataFrame
sample_data = pd.DataFrame({
    'age': age,
    'sex': sex,
    'cp': cp,
    'trestbps': trestbps,
    'chol': chol,
    'fbs': fbs,
    'restecg': restecg,
    'thalach': thalach,
    'exang': exang,
    'oldpeak': oldpeak,
    'slope': slope,
    'ca': ca,
    'thal': thal
})


In [ ]:
# Normalize the input data (using the same scaler fitted on training data)
input_data_scaled = scaler.transform(sample_data)

# Predict using WTA model
predictions_wta = model_wta.predict(input_data_scaled)
predicted_class_wta = tf.argmax(predictions_wta, axis=-1).numpy()

# Predict using LTA model
predictions_lta = model_lta.predict(input_data_scaled)
predicted_class_lta = tf.argmax(predictions_lta, axis=-1).numpy()

# Convert predictions to probabilities
probabilities_wta = tf.nn.softmax(predictions_wta).numpy()
probabilities_lta = tf.nn.softmax(predictions_lta).numpy()

# Compute accuracy
def compute_accuracy(predictions, true_labels):
    return np.mean(predictions == true_labels)

# Assuming you have true labels for validation (e.g., y_test)
# Here, true_labels are just placeholders and need actual values for accurate computation
true_labels = np.random.randint(0, 2, size=num_samples)  # Replace with actual true labels

accuracy_wta = compute_accuracy(predicted_class_wta, true_labels)
accuracy_lta = compute_accuracy(predicted_class_lta, true_labels)

# Print results
print(f'Accuracy of WTA Model: {accuracy_wta * 100:.2f}%')
print(f'Accuracy of LTA Model: {accuracy_lta * 100:.2f}%')

# Display sample probabilities
sample_probabilities_wta = pd.DataFrame(probabilities_wta, columns=[f'Class_{i}' for i in range(probabilities_wta.shape[1])])
sample_probabilities_lta = pd.DataFrame(probabilities_lta, columns=[f'Class_{i}' for i in range(probabilities_lta.shape[1])])

print("Sample probabilities from WTA model:")
print(sample_probabilities_wta.head())

print("Sample probabilities from LTA model:")
print(sample_probabilities_lta.head())


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy of WTA Model: 31.70%
Accuracy of LTA Model: 34.30%
Sample probabilities from WTA model:
    Class_0   Class_1   Class_2   Class_3   Class_4
0  0.337898  0.193213  0.156720  0.156399  0.155770
1  0.220764  0.159534  0.171610  0.288859  0.159232
2  0.375820  0.156303  0.162525  0.153079  0.152274
3  0.401385  0.150831  0.149279  0.149254  0.149252
4  0.150215  0.149347  0.150131  0.401007  0.149301
Sample probabilities from LTA model:
    Class_0   Class_1   Class_2   Class_3   Class_4
0  0.194816  0.206365  0.217610  0.201973  0.179237
1  0.189594  0.212498  0.189034  0.215636  0.193238
2  0.191090  0.202981  0.200487  0.214480  0.190961
3  0.190625  0.192798  0.250395  0.194174  0.172008
4  0.190468  0.220150  0.194267  0.200097  0.195017
